# Comparing Energies between NCMC and instaneous switching
The `NCMC` function within the `SaltSwap` class uses a velocity verlet (VV) integrator, which is a constant energy integrator. Thus, when the number of NCMC kernals is 1, the energies from instaneous switching and NCMC should be identical, up to the precision of the VV integrator.

## 1. Number of VV steps = 1

In [1]:
# Testing the addition of NCMC moves in saltwap.
# Importing:
from datetime import datetime
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
import sys
sys.path.append("../saltswap/")
import saltswap

# Defining constants.
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure = 1*unit.atmospheres
temperature = 300*unit.kelvin
delta_chem = -100*unit.kilojoule_per_mole

#Loading a premade water box:
pdb = app.PDBFile('../examples/waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

# Minimizing energy with a temporary integrator.
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)
openmm.LocalEnergyMinimizer.minimize(context, 1.0, 25)
positions = context.getState(getPositions=True).getPositions(asNumpy=True)
del context, integrator

# Defining the new system integrator that will be used for the standard MD moves:
integrator = openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(pressure, temperature, 25))
# Initialising the constant salt class. Inside, a 'compoundintegrator' is defined to do NCMC
#     NCMC parameters:
nkernals = 1           # Number of NCMC steps
nverlet = 1            # Number of velocity verlet steps in each NCMC step     
mc_saltswap = saltswap.SaltSwap(system=system,topology=pdb.topology,temperature=temperature,delta_chem=delta_chem,
                                integrator=integrator,pressure=pressure,
                                debug=True,nkernals=nkernals, nverlet_steps=nverlet)

# Creating the working the context
platform = openmm.Platform.getPlatformByName('CPU')
context = openmm.Context(system, mc_saltswap.compound_integrator,platform)
context.setPositions(positions)

# Brief thermalisation:
context.setVelocitiesToTemperature(temperature)
integrator.step(10)
positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)

# Cycles of MD and MC salt exchanges:
iterations = 10         # Number of rounds of MD and constant salt moves
nsteps = 50           # Amount of MD steps per iteration. 250000 steps = 500 picoseconds
nattempts = 5        # Number of identity exchanges attempts per iteration 

startTime = datetime.now()
for i in range(iterations):
    integrator.step(nsteps)
    mc_saltswap.update(context,nattempts=nattempts)
    positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)
tm1 = datetime.now() - startTime

identifyResidues: 501 HOH molecules identified.


In [2]:
vv1_diff = np.mean(np.absolute(np.array(mc_saltswap.nrg_isnt) - np.array(mc_saltswap.nrg_ncmc)))
print "Average difference between enegies =", vv1_diff
print "Mean time (s) per move = %.1f" %(1.0*tm1.seconds/nattempts/iterations)

Average difference between enegies = 2.71606445313e-05 kJ/mol
Mean time (s) per move = 0.1


## 2. Number of VV steps = 5

In [3]:
#Loading a premade water box:
pdb = app.PDBFile('../examples/waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

# Minimizing energy with a temporary integrator.
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)
openmm.LocalEnergyMinimizer.minimize(context, 1.0, 25)
positions = context.getState(getPositions=True).getPositions(asNumpy=True)
del context, integrator

# Defining the new system integrator that will be used for the standard MD moves:
integrator = openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(pressure, temperature, 25))
# Initialising the constant salt class. Inside, a 'compoundintegrator' is defined to do NCMC
#     NCMC parameters:
nkernals = 1           # Number of NCMC steps
nverlet = 5            # Number of velocity verlet steps in each NCMC step     
mc_saltswap = saltswap.SaltSwap(system=system,topology=pdb.topology,temperature=temperature,delta_chem=delta_chem,
                                integrator=integrator,pressure=pressure,
                                debug=True,nkernals=nkernals, nverlet_steps=nverlet)

# Creating the working the context
platform = openmm.Platform.getPlatformByName('CPU')
context = openmm.Context(system, mc_saltswap.compound_integrator,platform)
context.setPositions(positions)

# Brief thermalisation:
context.setVelocitiesToTemperature(temperature)
integrator.step(10)
positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)

# Cycles of MD and MC salt exchanges:
iterations = 10         # Number of rounds of MD and constant salt moves
nsteps = 50           # Amount of MD steps per iteration. 250000 steps = 500 picoseconds
nattempts = 5        # Number of identity exchanges attempts per iteration 

startTime = datetime.now()
for i in range(iterations):
    integrator.step(nsteps)
    mc_saltswap.update(context,nattempts=nattempts)
    positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)
tm5 = datetime.now() - startTime


identifyResidues: 501 HOH molecules identified.


In [4]:
vv5_diff = np.mean(np.absolute(np.array(mc_saltswap.nrg_isnt) - np.array(mc_saltswap.nrg_ncmc)))
print "Average difference between enegies =", vv1_diff
print "Mean time (s) per move = %.1f" %(1.0*tm5.seconds/nattempts/iterations)

Average difference between enegies = 2.71606445313e-05 kJ/mol
Mean time (s) per move = 0.1


## 2. Number of VV steps = 25

In [5]:
#Loading a premade water box:
pdb = app.PDBFile('../examples/waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

# Minimizing energy with a temporary integrator.
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)
openmm.LocalEnergyMinimizer.minimize(context, 1.0, 25)
positions = context.getState(getPositions=True).getPositions(asNumpy=True)
del context, integrator

# Defining the new system integrator that will be used for the standard MD moves:
integrator = openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(pressure, temperature, 25))
# Initialising the constant salt class. Inside, a 'compoundintegrator' is defined to do NCMC
#     NCMC parameters:
nkernals = 1           # Number of NCMC steps
nverlet = 25            # Number of velocity verlet steps in each NCMC step     
mc_saltswap = saltswap.SaltSwap(system=system,topology=pdb.topology,temperature=temperature,delta_chem=delta_chem,
                                integrator=integrator,pressure=pressure,
                                debug=True,nkernals=nkernals, nverlet_steps=nverlet)

# Creating the working the context
platform = openmm.Platform.getPlatformByName('CPU')
context = openmm.Context(system, mc_saltswap.compound_integrator,platform)
context.setPositions(positions)

# Brief thermalisation:
context.setVelocitiesToTemperature(temperature)
integrator.step(10)
positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)

# Cycles of MD and MC salt exchanges:
iterations = 10         # Number of rounds of MD and constant salt moves
nsteps = 50           # Amount of MD steps per iteration. 250000 steps = 500 picoseconds
nattempts = 5        # Number of identity exchanges attempts per iteration 

startTime = datetime.now()
for i in range(iterations):
    integrator.step(nsteps)
    mc_saltswap.update(context,nattempts=nattempts)
    positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)
tm25 = datetime.now() - startTime

identifyResidues: 501 HOH molecules identified.


In [6]:
vv20_diff = np.mean(np.absolute(np.array(mc_saltswap.nrg_isnt) - np.array(mc_saltswap.nrg_ncmc)))
print "Average difference between enegies =", vv1_diff
print "Mean time (s) per move = %.1f" %(1.0*tm25.seconds/nattempts/iterations)

Average difference between enegies = 2.71606445313e-05 kJ/mol
Mean time (s) per move = 0.4


## 3. Number of VV steps = 100

In [7]:
#Loading a premade water box:
pdb = app.PDBFile('../examples/waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

# Minimizing energy with a temporary integrator.
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)
openmm.LocalEnergyMinimizer.minimize(context, 1.0, 25)
positions = context.getState(getPositions=True).getPositions(asNumpy=True)
del context, integrator

# Defining the new system integrator that will be used for the standard MD moves:
integrator = openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(pressure, temperature, 25))
# Initialising the constant salt class. Inside, a 'compoundintegrator' is defined to do NCMC
#     NCMC parameters:
nkernals = 1           # Number of NCMC steps
nverlet = 100            # Number of velocity verlet steps in each NCMC step     
mc_saltswap = saltswap.SaltSwap(system=system,topology=pdb.topology,temperature=temperature,delta_chem=delta_chem,
                                integrator=integrator,pressure=pressure,
                                debug=True,nkernals=nkernals, nverlet_steps=nverlet)

# Creating the working the context
platform = openmm.Platform.getPlatformByName('CPU')
context = openmm.Context(system, mc_saltswap.compound_integrator,platform)
context.setPositions(positions)

# Brief thermalisation:
context.setVelocitiesToTemperature(temperature)
integrator.step(10)
positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)

# Cycles of MD and MC salt exchanges:
iterations = 10         # Number of rounds of MD and constant salt moves
nsteps = 50           # Amount of MD steps per iteration. 250000 steps = 500 picoseconds
nattempts = 5        # Number of identity exchanges attempts per iteration 

startTime = datetime.now()
for i in range(iterations):
    integrator.step(nsteps)
    mc_saltswap.update(context,nattempts=nattempts)
    positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)
tm100 = datetime.now() - startTime

identifyResidues: 501 HOH molecules identified.


In [8]:
vv20_diff = np.mean(np.absolute(np.array(mc_saltswap.nrg_isnt) - np.array(mc_saltswap.nrg_ncmc)))
print "Average difference between enegies =", vv1_diff
print "Mean time (s) per move = %.1f" %(1.0*tm100.seconds/nattempts/iterations)

Average difference between enegies = 2.71606445313e-05 kJ/mol
Mean time (s) per move = 1.3
